In [1]:
import json

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

2023-05-07 22:03:11.200513: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 22:03:11.397545: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-07 22:03:11.398839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-07 22:03:12.153222: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class schema:
    def __init__ (self, id):
        self.id = id
        self.tableNames = []

# Data Analysis and Preprocessing

We note that the databases with the largest amounts of queries in Spider are college_2/college_1, hr_1, store_1, and soccer_2.
We will create 4 models applied on each of these databases.

In [3]:
# Loading Spider Data
with open('./spider-data/tables.json', 'r') as f:
  schemaDoc = json.load(f)

with open('./spider-data/train_spider.json', 'r') as f:
  queryDoc = json.load(f)

schemas = {}
db_counts = {}

for idx, iquery in enumerate(queryDoc):
    db_counts[iquery['db_id']] = db_counts.get(iquery['db_id'], 0) + 1

print(sorted(db_counts.items(), key=lambda x:x[1], reverse=True))

[('college_2', 170), ('college_1', 164), ('hr_1', 124), ('store_1', 112), ('soccer_2', 106), ('bike_1', 104), ('music_1', 100), ('hospital_1', 100), ('music_2', 100), ('dorm_1', 100), ('allergy_1', 98), ('movie_1', 98), ('flight_1', 96), ('driving_school', 93), ('cre_Doc_Tracking_DB', 90), ('department_store', 88), ('customers_and_addresses', 88), ('activity_1', 88), ('network_2', 86), ('products_gen_characteristics', 86), ('game_1', 86), ('chinook_1', 84), ('cre_Theme_park', 84), ('cre_Docs_and_Epenses', 84), ('customers_and_invoices', 82), ('sakila_1', 82), ('baseball_1', 82), ('e_learning', 82), ('cre_Drama_Workshop_Groups', 82), ('wine_1', 82), ('flight_4', 82), ('customers_card_transactions', 80), ('apartment_rentals', 80), ('formula_1', 80), ('loan_1', 80), ('manufactory_1', 80), ('tracking_grants_for_research', 78), ('inn_1', 74), ('college_3', 74), ('voter_2', 72), ('csu_1', 70), ('club_1', 70), ('election', 68), ('student_1', 68), ('icfp_1', 66), ('music_4', 60), ('tracking_or

### Filter Queries

We now filter the query and schema doc entries to the queries linked to the four databases we're creating models for (college_2, hr_1, store_1, and soccer_2)

In [8]:
filteredQueryDoc = [query for query in queryDoc if query['db_id'] in ['college_2','hr_1','store_1','soccer_2']]
filteredSchemaDoc = [schema for schema in schemaDoc if schema['db_id'] in ['college_2','hr_1','store_1','soccer_2']]

### Create outputs

Now we create our outputs (y) for the model. The outputs are in the following format:

e.g. `[0,0,0,1,0,1,1,...]`

Where the *nth* index corresponds to the *nth* table in the schema. The value contains a 1 if the table is referenced in the SQL query, and a 0 if it is not.

In [9]:
for idx, iSchema in enumerate(filteredSchemaDoc):
    dbid = iSchema['db_id']
    test = schema(dbid)
    for columnName in iSchema['table_names_original']:
        test.tableNames.append(columnName)
    print("Schema", idx, ":", test.id, ":", test.tableNames)
    schemas[dbid] = test

tables = []
nl_queries = []

for idx, iquery in enumerate(filteredQueryDoc):
   # Initialize a blank array with the length of the number of tables
   tablesArray = [0] * len(schemas[iquery['db_id']].tableNames)
   # Get all the tokens from teh particular query we are looking at
   queryTokens = iquery['query_toks']

   # Search through all the tokens in the given query
   for idx, queryToken in enumerate(queryTokens):
      # Only look at tokens that come after the word "from," since those are the table names
      if (idx > 0) & (queryTokens[idx - 1].lower() == "from") | (queryTokens[idx - 1].lower() == "join"):
        # Search through all of the table names in the database we are looking at

        # TODO: Use spacy to tag all table names using named entity recognition (NER).
        for jdx, tableName in enumerate(schemas[iquery['db_id']].tableNames):
           # If the table names match, indicate it in tablesArray
           if tableName.lower() == queryToken.lower():
              tablesArray[jdx] = 1
   
   tables.append(tablesArray)
   print(tablesArray)
   #print(schemas[iquery['db_id']].id)
   print(iquery['query_toks'])

Schema 0 : college_2 : ['classroom', 'department', 'course', 'instructor', 'section', 'teaches', 'student', 'takes', 'advisor', 'time_slot', 'prereq']
Schema 1 : hr_1 : ['regions', 'countries', 'departments', 'jobs', 'employees', 'job_history', 'locations']
Schema 2 : store_1 : ['artists', 'sqlite_sequence', 'albums', 'employees', 'customers', 'genres', 'invoices', 'media_types', 'tracks', 'invoice_lines', 'playlists', 'playlist_tracks']
Schema 3 : soccer_2 : ['College', 'Player', 'Tryout']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['SELECT', 'billing_country', ',', 'COUNT', '(', '*', ')', 'FROM', 'invoices', 'GROUP', 'BY', 'billing_country', 'ORDER', 'BY', 'count', '(', '*', ')', 'DESC', 'LIMIT', '5', ';']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['SELECT', 'billing_country', ',', 'COUNT', '(', '*', ')', 'FROM', 'invoices', 'GROUP', 'BY', 'billing_country', 'ORDER', 'BY', 'count', '(', '*', ')', 'DESC', 'LIMIT', '5', ';']
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
['SELECT', 'billing_country', ',', 

In [5]:
def loadTrainDF(inputLines):
    wordSense = 0
    questions = []
    tables = []

    for line in inputLines:
        if line == "1\n":
            wordSense = 1
        elif line == "2\n":
            wordSense = 2
        elif (line != "\n") & (wordSense != 0):
            lines.append(line.strip())
            senses.append(wordSense)
    trainDF = pd.DataFrame(data={'text': lines,'sense': senses})
    trainDF['classification'] = trainDF['sense'].apply(lambda x: x - 1)
    return trainDF

In [ ]:
def get_sentence_embeddings(bert_preprocess_model, bert_encoder_model, sentences = []):
    
    text_preprocessed = bert_preprocess_model(sentences)
    return bert_encoder_model(text_preprocessed)['pooled_output']

In [ ]:
# Code Adopted and modified from YouTube video: https://www.youtube.com/watch?v=7kLi8u2dJz0&t=719s
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [ ]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
# ! Code Adopted and Modified from https://github.com/eclarson/MachineLearningNotebooks and https://www.youtube.com/watch?v=hOCDJyZ6quA
# Notebook 13: RNN Basics
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input

# BERT Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# NN Layers
l = tf.keras.layers.Dropout(0.2, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# Final Model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=3)
model.save('rubbish')

In [ ]:
rubbish_model = tf.keras.models.load_model('rubbish')

In [ ]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf

stopWordSet = nltk.corpus.stopwords.words('english')
stemmer = nltk.SnowballStemmer(language='english')
tokenizer = nltk.RegexpTokenizer(r"[a-zA-Z0-9]+")

def getTestData(inputfile):
    wordSense = 0
    lines = []
    senses = []

    for line in inputLines:
        if line == "1\n":
            wordSense = 1
        elif line == "2\n":
            wordSense = 2
        elif (line != "\n") & (wordSense != 0):
            lines.append(line.strip())
            senses.append(wordSense)
    trainDF = pd.DataFrame(data={'text': lines,'sense': senses})
    return trainDF

rubbishTestInput = open("testing/rubbish.txt", "r", encoding='utf-8')

lines = rubbishTestInput.readlines()
linesStripped = np.asarray([line.strip() for line in lines])

rubbish_predictions = rubbish_model.predict(linesStripped)
print(rubbish_predictions)

output_file = open('results/result_rubbish_Michael_Lennon.txt', "w", encoding='utf-8')

for prediction in rubbish_predictions:
    if prediction > 0.5:
        output_file.write('2\n')
    else:
        output_file.write('1\n')


# Rubbish Model.train()
rubbishTestInput.close()
output_file.close()